In [1]:
import pandas as pd

In [8]:
test_data = pd.read_csv('./data/filled_test_set.csv')
test_data = test_data.drop(columns=['score'])
print(test_data.describe())
nan_data_info = pd.DataFrame({'NaN Count': test_data.isna().sum(), 'Data Type': test_data.dtypes})
print(nan_data_info)

            srch_id       site_id  visitor_location_country_id  \
count  4.959183e+06  4.959183e+06                 4.959183e+06   
mean   1.504747e-06  8.979214e+00                 1.641250e+02   
std    8.682007e-07  7.667722e+00                 6.158367e+01   
min    0.000000e+00  0.000000e+00                 0.000000e+00   
25%    7.522652e-07  4.000000e+00                 9.400000e+01   
50%    1.508801e-06  4.000000e+00                 2.050000e+02   
75%    2.257098e-06  1.300000e+01                 2.050000e+02   
max    3.004943e-06  3.300000e+01                 2.160000e+02   

       visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  \
count             4.959183e+06          4.959183e+06     4.959183e+06   
mean              1.220584e-01          2.851837e-05     1.277441e+02   
std               7.859750e-03          4.679312e-06     5.042443e+01   
min               0.000000e+00          0.000000e+00     0.000000e+00   
25%               1.222364e-01          

In [9]:
import joblib
# test_set = test_data.drop(columns=['prop_id', 'site_id', 'random_bool', 'random_bool', 'srch_destination_id', 'visitor_location_country_id'])
test_set = test_data
model = joblib.load('models/lightgbm.pkl')

In [10]:
qids = test_set.groupby("srch_id")["srch_id"].count().to_numpy()
X = test_set.drop(["srch_id"], axis=1)
print(X)

         site_id  visitor_location_country_id  visitor_hist_starrating  \
0             23                          202                 0.139662   
1             23                          202                 0.122236   
2             23                          202                 0.119165   
3             23                          202                 0.139662   
4             23                          202                 0.142733   
...          ...                          ...                      ...   
4959178       23                          202                 0.122236   
4959179       23                          202                 0.122236   
4959180       23                          202                 0.122236   
4959181       23                          202                 0.122236   
4959182       23                          202                 0.122236   

         visitor_hist_adr_usd  prop_country_id  prop_id  prop_starrating  \
0                    0.000028      

In [11]:
output = model.predict(X)
print(output)

[ 0.67690453  0.81633603  0.78399558 ... -0.04064394 -0.06341746
  0.03369807]


In [12]:
result = test_data[['srch_id', 'prop_id']]
result['pred'] = output
print(result)

          srch_id  prop_id      pred
0        0.000000     2923  0.676905
1        0.000000     5107  0.816336
2        0.000000    13025  0.783996
3        0.000000    20577  0.857297
4        0.000000    22222  1.109803
...           ...      ...       ...
4959178  0.000003    29397  0.175627
4959179  0.000003    31190  0.260187
4959180  0.000003    32374 -0.040644
4959181  0.000003    86801 -0.063417
4959182  0.000003    91448  0.033698

[4959183 rows x 3 columns]


C:\Users\china\AppData\Local\Temp\ipykernel_14724\4200473951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['pred'] = output


In [13]:
sorted_df = result.groupby('srch_id').apply(lambda x: x.sort_values(by='pred', ascending=False))
sorted_df.reset_index(drop=True, inplace=True)

C:\Users\china\AppData\Local\Temp\ipykernel_14724\3395379698.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = result.groupby('srch_id').apply(lambda x: x.sort_values(by='pred', ascending=False))


In [14]:
print(sorted_df)

          srch_id  prop_id      pred
0        0.000000   113676  1.238137
1        0.000000    87072  1.238137
2        0.000000    67719  1.238137
3        0.000000    58701  1.224185
4        0.000000    68072  1.127538
...           ...      ...       ...
4959178  0.000003    29397  0.175627
4959179  0.000003    26663  0.160193
4959180  0.000003    91448  0.033698
4959181  0.000003    32374 -0.040644
4959182  0.000003    86801 -0.063417

[4959183 rows x 3 columns]


In [15]:
final_res = sorted_df[['srch_id', 'prop_id']]
final_res = final_res.astype(int)
print(final_res)
final_res.to_csv('./data/final_res.csv', index=False)

         srch_id  prop_id
0              0   113676
1              0    87072
2              0    67719
3              0    58701
4              0    68072
...          ...      ...
4959178        0    29397
4959179        0    26663
4959180        0    91448
4959181        0    32374
4959182        0    86801

[4959183 rows x 2 columns]
